Import packages

In [1]:
#
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

# Some settings

In [2]:
#
n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

# Load data

Load mnist data

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets('datasets/mnist')

Extracting datasets/mnist/train-images-idx3-ubyte.gz
Extracting datasets/mnist/train-labels-idx1-ubyte.gz
Extracting datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting datasets/mnist/t10k-labels-idx1-ubyte.gz


In [5]:
X_train = mnist.train.images
y_train = mnist.train.labels

In [6]:
X_val = mnist.validation.images
y_val = mnist.validation.labels

# Network construction

Define nodes for predictor X and target y

In [7]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')
training = tf.placeholder_with_default(False, (), name='training')

In the namscope "dnn", define a network of:
1. dense layer1 of n_hidden1 neurons
2. batch normalization layer of momentum = 0.9
3. ELU activation function
4. dense layer2 of n_hidden2 neurons
5. batch normalization layer of momentum = 0.9
6. ELU activation function
7. dense layer of n_outputs neurons
8. batch normalization layer of momnetus = 0.9

In [8]:
with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=None, name='hidden1')
    bn1 = tf.layers.batch_normalization(hidden1, momentum=0.9,
                                       training=training, name='batch_norm1')
    elu1 = tf.nn.elu(bn1, name='ELU1')
    
    hidden2 = tf.layers.dense(elu1, n_hidden2, activation=None, name='hidden2')
    bn2 = tf.layers.batch_normalization(hidden2, momentum=0.9,
                                       training=training, name='batch_norm2')
    elu2 = tf.nn.elu(bn2, name='ELU2')
    
    logits_before_bn = tf.layers.dense(elu2, n_hidden2, activation=None)
    logits = tf.layers.batch_normalization(logits_before_bn, momentum=0.9,
                                          training=training, name='outputs_norm')

In the namescope "loss", define the loss function

In [9]:
with tf.name_scope('loss'):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(entropy, name='loss')

In the namescope 'train', define the training operation

In [10]:
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss, name='training_op')

In the namesope 'eval', define the node of accuracy and the summary node to log accuracy

In [11]:
with tf.name_scope('eval'):
    accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logits, y, 1), tf.float32), name='accuracy')
    accuracy_summary = tf.summary.scalar('Accuracy', accuracy)

In [12]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
writer = tf.summary.FileWriter('logs/mnist_bn', graph=tf.get_default_graph())

Execution phase: 
1. run 40 epoches, and use mini-batch size of 50
2. After each epoch, print out the validation accuracy
3. Save the final model to tmp/mnist_BN.ckpt
4. Save log to visualize the graph and test accuracy learning rate

In [13]:
n_epoch = 40
batch_size = 50
n_step = int(np.ceil(mnist.train.num_examples / batch_size))

In [14]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

In [15]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoch):
        for step in range(n_step):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run([training_op, extra_update_ops], feed_dict={training:True, X: X_batch, y: y_batch})
        [accuracy_val, sum_val] = sess.run([accuracy, accuracy_summary], feed_dict={X: X_val, y: y_val})
        print('Epoch: {}, Validation accuracy: {:.4f}'.format(epoch, accuracy_val))
        writer.add_summary(sum_val, global_step=batch_size * epoch + step)
    saver.save(sess, 'tmp/mnist_BN.ckpt')
    writer.close()

Epoch: 0, Validation accuracy: 0.9646
Epoch: 1, Validation accuracy: 0.9704
Epoch: 2, Validation accuracy: 0.9748
Epoch: 3, Validation accuracy: 0.9766
Epoch: 4, Validation accuracy: 0.9798
Epoch: 5, Validation accuracy: 0.9786
Epoch: 6, Validation accuracy: 0.9800
Epoch: 7, Validation accuracy: 0.9800
Epoch: 8, Validation accuracy: 0.9780
Epoch: 9, Validation accuracy: 0.9816
Epoch: 10, Validation accuracy: 0.9820
Epoch: 11, Validation accuracy: 0.9824
Epoch: 12, Validation accuracy: 0.9796
Epoch: 13, Validation accuracy: 0.9830
Epoch: 14, Validation accuracy: 0.9818
Epoch: 15, Validation accuracy: 0.9818
Epoch: 16, Validation accuracy: 0.9836
Epoch: 17, Validation accuracy: 0.9848
Epoch: 18, Validation accuracy: 0.9840
Epoch: 19, Validation accuracy: 0.9836
Epoch: 20, Validation accuracy: 0.9836
Epoch: 21, Validation accuracy: 0.9830
Epoch: 22, Validation accuracy: 0.9814
Epoch: 23, Validation accuracy: 0.9822
Epoch: 24, Validation accuracy: 0.9814
Epoch: 25, Validation accuracy: 0.9

Now visualize the graph and the validation accuracy learning curve in TensorBoard